# Práctico 6: Campos receptivos

El siguiente artículo va a visualizar algunos modelos matemáticos de la actividad de las células cerebrales.
En algunas regiones del cerebro, las neuronas son excitadas o inhibidas por neuronas de una capa de entrada anterior. A ese conjunto se lo llama campo receptivo de esa neurona.

Como el área visual utiliza los campos receptivos como detectores de características (como la detección de bordes y de la orientación de los bordes) para imágenes naturales, se puede examinar muy bien la aplicación de distintas funciones de campos receptivos sobre imágenes.

## Configuración

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy as sp
import numpy as np
import imageio.v3 as iio
import ipywidgets as widgets

### Funciones utilitarias

In [ ]:
def construir_filtro(func, h=30):
    """Construye un filtro 2D de tamaño hxh aplicando func(x, y) en una grilla centrada.

    Args:
        func (func): función que recibe (x, y)
        h (int): tamaño del filtro (por defecto 30)

    Returns:
        ndarray: matriz 2D
    """
    g = np.zeros((h, h))
    for xi in range(h):
        for yi in range(h):
            x = xi - h / 2
            y = yi - h / 2
            g[xi, yi] = func(x, y)
    return g

### Funciones de graficado

In [ ]:
def visualizar_imagen(img):
    plt.subplots(figsize=(6, 4))
    plt.imshow(img, cmap="gray")
    plt.show()
    
def visualizar_filtro2d(*fs):
    n = len(fs)
    fig, axes = plt.subplots(1, n, figsize=(3 * n, 3))
    
    if n == 1:
        axes = [axes]

    vmin = min(f.min() for f in fs)
    vmax = max(f.max() for f in fs)

    for i, (ax, f) in enumerate(zip(axes, fs)):
        ax.set_title(f"Filtro {i+1}")
        ax.imshow(f, cmap="coolwarm", vmin=vmin, vmax=vmax)
        ax.set_axis_off()

    plt.show()
    
def visualizar_convolucion2d(s, f, convolucion):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, width_ratios=[2, 1, 2], figsize=(12, 4))

    ax1.set_title("Imagen original")
    ax1.matshow(s, cmap="gray")
    ax1.set_axis_off()
    
    ax2.set_title("Filtro")
    ax2.imshow(f, cmap="coolwarm")
    ax2.set_axis_off()
    
    ax3.set_title("Resultado de la convolución")
    ax3.imshow(convolucion, cmap="gray")
    ax3.set_axis_off()
    
    fig.subplots_adjust(wspace=0.2)
    fig.text(0.420, 0.5, '+', ha='center', va='center', fontsize=16)
    fig.text(0.605, 0.5, '=', ha='center', va='center', fontsize=16)
    
    plt.show()

# Visualización de imágenes de ejemplo

Examinamos el efecto en las siguientes imágenes. En la vía visual, las imágenes pueden verse como la entrada desde la retina hacia áreas visuales superiores.

In [ ]:
barImg = iio.imread('https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico6_Bar.png')
visualizar_imagen(barImg)

In [ ]:
bugImg = iio.imread('https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico6_Stinkbug.png')
visualizar_imagen(bugImg)

## Funciones de campo receptivo

La función gaussiana bidimensional se usa en procesamiento de imágenes como filtro para desenfocar (blur).

$$\phi(x,y) = \frac{1}{2\pi\sigma^2}\exp{(-\frac{1}{2\pi\sigma^2}(x^2+ y^2))}$$

In [ ]:
def gaussian2D(x, y, sigma):
    return (1.0/(1*np.pi*(sigma**2)))*np.exp(-(1.0/(2*(sigma**2)))*(x**2 + y**2))

Como la función de convolución de SciPy no acepta funciones directamente, lo que hacemos es muestrear la función. Para eso, utilizaremos una funcion ya definida:

In [ ]:
help(construir_filtro)

Ejecutá la celda siguiente para visualizar el resultado del muestreo de la función gaussiana bidimensional. Más adelante, lo usaremos como filtro para hacer convoluciones sobre las imágenes de ejemplo.

In [ ]:
@widgets.interact(sigma=(1, 30))
def visualizar_filtro_gaussiano(sigma=6):
    f = construir_filtro(lambda x, y: gaussian2D(x,y,sigma))
    visualizar_filtro2d(f)

La función gaussiana es simétrica en forma circular, lo que produce la excitación de un píxel central a partir de los píxeles cercanos durante la convolución.

En el contexto de la transformada de Fourier, actúa como un filtro pasa bajos, que elimina las frecuencias altas en el dominio frecuencial de la imagen y, por lo tanto, genera un efecto de desenfoque.

La convolución es el proceso de aplicar el filtro sobre la entrada, que es la imagen $I(x,y)$ que denota el valor de gris del píxel en la posición especificada.

$$\int \int I(x',y')\phi(x-x',y-y')dx'dy'$$

Cuando se aplica el filtro gaussiano, cada neurona de la capa de salida es excitada por las neuronas cercanas de la imagen.

El resultado de la convolución también puede visualizarse en una imagen.

In [ ]:
@widgets.interact(sigma=(1, 30))
def simular_filtro_gaussiano(sigma=6):
    s = barImg
    f = construir_filtro(lambda x, y: gaussian2D(x, y, sigma))
    convolucion = sp.signal.convolve(s, f, mode='same')
    visualizar_convolucion2d(s, f, convolucion)

Ahora lo mismo pero sobre la imagen del insecto.

In [ ]:
@widgets.interact(sigma=(1, 30))
def simular_filtro_gaussiano(sigma=6):
    s = bugImg
    f = construir_filtro(lambda x, y: gaussian2D(x,y,sigma))
    convolucion = sp.signal.convolve(s, f, mode='same')
    visualizar_convolucion2d(s, f, convolucion)

## Diferencia de gaussianas

La función sombrero mexicano es una diferencia de gaussianas, que genera un campo receptivo con centro excitador y periferia inhibidora, típico de las células ganglionares de la retina o neuronas del LGN. Se puede considerar como un detector básico de bordes.

Ejecutá la celda siguiente para definir la función y luego visualizar el resultado de muestrearla.

In [ ]:
def mexicanHat(x,y,sigma1,sigma2):
    return gaussian2D(x,y,sigma1) - gaussian2D(x,y,sigma2)

@widgets.interact(sigma1=(1, 30), sigma2=(1, 30))
def visualizar_sombrero_mexicano(sigma1=4, sigma2=3):
    f = construir_filtro(lambda x, y: mexicanHat(x,y,sigma1, sigma2))
    visualizar_filtro2d(f)

Ahora veamos como es el resultado de realizar una convolución con este filtro sobre la imagen de la barra.

In [ ]:
@widgets.interact(sigma1=(1, 30), sigma2=(1, 30))
def simular_sombrero_mexicano(sigma1=4, sigma2=3):
    s = barImg
    f = construir_filtro(lambda x, y: mexicanHat(x, y, sigma1, sigma2))
    convolucion = sp.signal.convolve(s, f, mode='same')
    visualizar_convolucion2d(s, f, convolucion)

Y ahora sobre la imagen del insecto.

In [ ]:
@widgets.interact(sigma1=(1, 30), sigma2=(1, 30))
def simular_sombrero_mexicano(sigma1=4, sigma2=3):
    s = bugImg
    f = construir_filtro(lambda x, y: mexicanHat(x, y, sigma1, sigma2))
    convolucion = sp.signal.convolve(s, f, mode='same')
    visualizar_convolucion2d(s, f, convolucion)

## Funciones de Gabor

Las funciones de Gabor se usan para detectar bordes con una orientación específica en imágenes. Se encuentran neuronas que pueden modelarse con funciones de Gabor en todo el córtex visual.

Gabor impar:

$$g_s(x,y):=sin(\omega_x x + \omega_y y)\exp{(-\frac{x^2+y^2}{2\sigma^2})}$$

Gabor par:

$$g_c(x,y):=cos(\omega_x x + \omega_y y)\exp{(-\frac{x^2+y^2}{2\sigma^2})}$$

La orientación está determinada por la razón $\omega_y/\omega_x$.

La función $g_s$ se activa con bordes en forma de escalón, mientras que $g_c$ se activa con bordes en forma de línea.

In [ ]:
def gabor2d_impar(x, y, sigma, orientation):
    return np.sin(x + orientation*y) * np.exp(-(x**2 + y**2)/(2*sigma))

def gabor2d_par(x, y, sigma, orientation):
    return np.cos(x + orientation*y) * np.exp(-(x**2 + y**2)/(2*sigma))
    
@widgets.interact(sigma=(1, 30), orientation=(-3, 3, 0.1))
def visualizar_gabor_par(sigma=5, orientation=1):
    f1 = construir_filtro(lambda x, y: gabor2d_par(x, y, sigma, orientation))
    f2 = construir_filtro(lambda x, y: gabor2d_impar(x, y, sigma, orientation))
    visualizar_filtro2d(f1, f2)

In [ ]:
s = widgets.fixed(barImg)
sigma = widgets.IntSlider(value=5, min=1, max=30, description="σ")
orientation = widgets.FloatSlider(value=1, min=-3, max=3, description="orientación")

def simular_gabor_par(s, sigma, orientation):
    f = construir_filtro(lambda x,y: gabor2d_par(x, y, sigma, orientation))
    convolucion = sp.signal.convolve(s, f, mode='same')
    visualizar_convolucion2d(s, f, convolucion)

def simular_gabor_impar(s, sigma, orientation):
    f = construir_filtro(lambda x,y: gabor2d_impar(x, y, sigma, orientation))
    convolucion = sp.signal.convolve(s, f, mode='same')
    visualizar_convolucion2d(s, f, convolucion)

display(widgets.HBox([sigma, orientation]))
display(widgets.interactive_output(simular_gabor_par, {'s': s, 'sigma': sigma, 'orientation': orientation}))
display(widgets.interactive_output(simular_gabor_impar, {'s': s, 'sigma': sigma, 'orientation': orientation}))

Si hacemos lo mismo en la imagen del insecto podemos ver claramente las orientaciones de borde que excitan a la neurona.

In [ ]:
s = widgets.fixed(bugImg)
display(widgets.HBox([sigma, orientation]))
display(widgets.interactive_output(simular_gabor_par, {'s': s, 'sigma': sigma, 'orientation': orientation}))
display(widgets.interactive_output(simular_gabor_impar, {'s': s, 'sigma': sigma, 'orientation': orientation}))

## Combinación de filtros

Usando la imagen del campo receptivo centro-on, periferia-off como entrada del filtro de Gabor, obtenemos resultados diferentes.

In [ ]:
s = bugImg
f1 = construir_filtro(lambda x, y: mexicanHat(x, y, 2, 3))
f2 = construir_filtro(lambda x, y: gabor2d_impar(x, y, 5, 1))

primera_convolucion = sp.signal.convolve(s, f1, mode='same')
segunda_convolucion = sp.signal.convolve(primera_convolucion, f2, mode='same')

visualizar_convolucion2d(s, f1, primera_convolucion)
visualizar_convolucion2d(primera_convolucion, f2, segunda_convolucion)

## Pares en cuadratura

Una célula compleja puede responder igual de bien a bordes en forma de escalón y a líneas de cualquier polaridad. Esto se modela sumando las respuestas al cuadrado de los filtros de Gabor impar y par.

Graficamos abajo la respuesta de cada filtro lineal en cuadratura (similar a células simples de corteza visual primaria) a la imagen de la barra.

In [ ]:
s = barImg
f1 = construir_filtro(lambda x, y: gabor2d_impar(x, y, 10, 0))
f2 = construir_filtro(lambda x, y: gabor2d_par(x, y, 10, 0))

primera_convolucion = sp.signal.convolve(s, f1, mode='same')
segunda_convolucion = sp.signal.convolve(primera_convolucion, f2, mode='same')

visualizar_convolucion2d(s, f1, primera_convolucion)
visualizar_convolucion2d(primera_convolucion, f2, segunda_convolucion)

Ahora obtenemos la suma de los cuadrados de los filtros (filtro de energía, similar a células complejas) y graficamos la salida.

In [ ]:
# Sumamos las salidas de los filtros al cuadrado
cuadratura = np.square(primera_convolucion) + np.square(segunda_convolucion)

# Mostramos la imagen
visualizar_imagen(cuadratura)

A continuación, hacemos lo mismo para la foto del insecto:

In [ ]:
s = bugImg
f1 = construir_filtro(lambda x, y: gabor2d_impar(x, y, 10, 0))
f2 = construir_filtro(lambda x, y: gabor2d_par(x, y, 10, 0))

primera_convolucion = sp.signal.convolve(s, f1, mode='same')
segunda_convolucion = sp.signal.convolve(primera_convolucion, f2, mode='same')

visualizar_convolucion2d(s, f1, primera_convolucion)
visualizar_convolucion2d(primera_convolucion, f2, segunda_convolucion)

Finalmente, obtenemos nuevamente la suma de los cuadrados de los filtros y graficamos la salida.

In [ ]:
cuadratura = np.square(primera_convolucion) + np.square(segunda_convolucion)
visualizar_imagen(cuadratura)

## Bonificación 1: Ilusión de la rejilla de Hermann

In [ ]:
hermannImg = iio.imread('https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico6_Hermann.png')
visualizar_imagen(hermannImg)

In [ ]:
@widgets.interact(sigma1=(1, 30), sigma2=(1, 30))
def simular_sombrero_mexicano(sigma1=3, sigma2=4):
    s = hermannImg
    f = construir_filtro(lambda x, y: mexicanHat(x, y, sigma1, sigma2))
    convolucion = sp.signal.convolve(s, f, mode='same')
    visualizar_convolucion2d(s, f, convolucion)

## Bonificación 2: Bandas de Mach

In [ ]:
machImg = iio.imread('https://raw.githubusercontent.com/MaestriaCienciasCognitivas/ncc/main/book/static/Practico6_Mach.png')
visualizar_imagen(machImg)

In [ ]:
@widgets.interact(sigma1=(1, 30), sigma2=(1, 30))
def simular_sombrero_mexicano(sigma1=3, sigma2=4):
    s = machImg
    f = construir_filtro(lambda x, y: mexicanHat(x, y, sigma1, sigma2))
    convolucion = sp.signal.convolve(s, f, mode='same')
    visualizar_convolucion2d(s, f, convolucion)